# Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re
import os
import html
import matplotlib.pyplot as plt
import seaborn as sb

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Phrases
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from xgboost import XGBClassifier

r = RandomForestClassifier()
gbc = GradientBoostingClassifier()
d = DecisionTreeClassifier()
log = LogisticRegression()
k = KNeighborsClassifier()
g = GaussianNB()
b = BernoulliNB()
xgbc = XGBClassifier()
nb = MultinomialNB()

# EDA

In [ ]:
train=pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip",'\t')
test=pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip","\t")
att=pd.read_csv("../input/imdb-review-dataset/imdb_master.csv")

In [ ]:
att=att[["review","label"]]
att.columns=["review","sentiment"]
att=att[att.sentiment != "unsup"]
att=att.replace({"neg":0,"pos":1})
att.head()

In [ ]:
train.shape, test.shape, att.shape

In [ ]:
df=train.append(att, ignore_index=True)
df=df.append(test, ignore_index=True)
df.info()

In [ ]:
df.head()

# Clean Data

In [ ]:
df.review=html.unescape(df.review)
# removing html symbols

df.review=df.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
.str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
.str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['review']=df['review'].apply(lambda x: remove_emoji(x))

In [ ]:
from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

df["review"] = df["review"].apply(remove_accent)

In [ ]:
df.head()

In [ ]:
df.sentiment.value_counts()

# Tokenization

In [ ]:
corpora = df["review"].values
tokenized = [word_tokenize(corpus) for corpus in corpora]

print(tokenized[2222])

In [ ]:
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized]

print(tokenized[2222])

# Word Embedding for Feature Engineering

In [ ]:
%%time
from gensim.models import word2vec
np.set_printoptions(suppress=True)

feature_size = 256
context_size = 5
min_word = 1

word_vec= word2vec.Word2Vec(tokenized, vector_size=feature_size, \
                            window=context_size, min_count=min_word, \
                            epochs=50, seed=42)

# Final Dataframe

In [ ]:
word_vec_unpack = [(word, idx) for word, idx in \
                   word_vec.wv.key_to_index.items()]

tokens, indexes = zip(*word_vec_unpack)

word_vec_df = pd.DataFrame(word_vec.wv.vectors[indexes, :], index=tokens)

word_vec_df.head()

In [ ]:
tokenized_array = np.array(tokenized)

model_array = np.array([word_vec_df.loc[doc].mean(axis=0) for doc in tokenized_array])

In [ ]:
model_df = pd.DataFrame(model_array)
model_df["sentiment"] = df["sentiment"]

model_df.head()

In [ ]:
df_train=model_df[:75000]
df_test=model_df[75000:]

In [ ]:
y=df_train["sentiment"]
del df_train["sentiment"]
text=df_train

In [ ]:
del df_test["sentiment"]
test=df_test

In [ ]:
text.shape, y.shape, test.shape

# Model & Predict

In [ ]:
r.fit(text,y)
pred=r.predict(test)
pred = pred.astype(int)
pred

In [ ]:
sub=pd.DataFrame()
test_df=df[75000:]
sub["id"]=test_df["id"]
sub["sentiment"]=pred
sub
sub.to_csv("submission.csv", index = False)